## Determine metrics for timestamped trajectory evaluation

For trajectory information, we have spatial ground truth but **no** temporal ground truth. This is not surprising since the time taken to travel the same route on different days can be different based on the traffic conditions, train delays, etc. We capture high quality (high accuracy/high frequency/always on) temporal data from two different phone during the travel, and mark the temporal ground truth for the start and end of travel segments (trip/section) on the phone.

In this notebook, we explore the various options to create a _reference_ trajectory using the spatial ground truth and the timestamped accuracy control trajectories from android and iOS. We can then use the reference trajectory to evaluate the spatio-temporal accuracy of the experiment trajectories.

#### Import all required modules

In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev
import pandas as pd

In [ ]:
import emeval.metrics.reference_trajectory as emr

In [ ]:
# For computation
import numpy as np
import math
import functools
import scipy.stats as stats
import matplotlib.pyplot as plt

In [ ]:
import geopandas as gpd
import shapely as shp
import arrow

In [ ]:
import folium
import branca.element as bre
import geojson as gj
import copy

In [ ]:
import anytree as at
import anytree.importer as ati

#### Load all data

In [ ]:
DATASTORE_URL = "http://cardshark.cs.berkeley.edu"
AUTHOR_EMAIL = "shankari@eecs.berkeley.edu"
sd_la = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "unimodal_trip_car_bike_mtv_la")
sd_sj = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "car_scooter_brex_san_jose")
sd_ucb = eisd.SpecDetails(DATASTORE_URL, AUTHOR_EMAIL, "train_bus_ebike_mtv_ucb")

In [ ]:
import importlib
importlib.reload(ezpv)

In [ ]:
pv_la = eipv.PhoneView(sd_la)

In [ ]:
pv_sj = eipv.PhoneView(sd_sj)

In [ ]:
pv_ucb = eipv.PhoneView(sd_ucb)

### Test trajectories

We select four different runs as our deep-dive dataset to explore various reference trajectory creation options. These are:
1. `easiest`: overground, unimodal trip by car and bike where both control phones had very little error
1. `weird_android`: different run of trip (1) where the android control phone had very poor quality data even though we requested high quality data
1. `temporal_zigzags`: this trip is really the motivation for spatio-temporal accuracy. It is an underground trip with zig zags a prior point along the trip, classic example of why spatio-temporal accuracy is needed
1. `backtracking`: trip that includes a short u-turn + backtrack to ensure that real zigzags are respected

In [ ]:
deep_dive_dataset = {
    "easiest": {
        "ground_truth": {
            "leg": sd_la.get_ground_truth_for_leg("suburb_city_driving_weekend", "suburb_city_driving_weekend")
        },
        "temporal_control": {
            "android": pv_la.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][0],
            "ios": pv_la.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][0]
        }
    },
    "weird_android": {
        "ground_truth": {
            "leg": sd_la.get_ground_truth_for_leg("suburb_city_driving_weekend", "suburb_city_driving_weekend")
        },
        "temporal_control": {
            "android": pv_la.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0],
            "ios": pv_la.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]
        }
    },
    "temporal_zigzags": {
        "ground_truth": {
            "leg": sd_ucb.get_ground_truth_for_leg("mtv_to_berkeley_sf_bart", "subway_underground")
        },
        "temporal_control": {
            "android": pv_ucb.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]["evaluation_section_ranges"][5],
            "ios": pv_ucb.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][0]["evaluation_section_ranges"][5]
        }
    },
    "backtracking": {
        "ground_truth": {
            "leg": sd_sj.get_ground_truth_for_leg("bus trip with e-scooter access", "city_escooter")            
        },
        "temporal_control": {
            "android": pv_sj.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][1]["evaluation_section_ranges"][1],
            "ios": pv_sj.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][0]["evaluation_trip_ranges"][1]["evaluation_section_ranges"][1]
        }
    }
}

In [ ]:
import importlib
importlib.reload(eisd)

In [ ]:
def add_section_and_points(curr_map, eval_section, loc_df_label, layer_name, disp_color, with_points=False):
    sensed_location_df = eval_section[loc_df_label]
    print("Adding section for %s with length %s" % (layer_name, len(sensed_location_df)))
    sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(sensed_location_df.longitude, sensed_location_df.latitude))),
                                   properties={"style": {"color": disp_color}, "ts": list(sensed_location_df.ts)})
    sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="sensed_values (%s)" % layer_name)
    curr_map.add_child(sensed_leg_gj_feature)
    if with_points:
        sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="sensed_points (%s)" % layer_name,
                                                      color=disp_color, tz="America/Los_Angeles")
        curr_map.add_child(sensed_leg_gj_points)

def display_gt_and_controls(entry, loc_df_label, with_points=False):
    curr_map = folium.Map()
    print("Using ground truth %s" % entry["ground_truth"]["leg"]["id"])
    gt_leg_gj = eisd.SpecDetails.get_geojson_for_leg(entry["ground_truth"]["leg"])
    gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
    curr_map.add_child(gt_leg_gj_feature)
    if with_points:
        gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
        curr_map.add_child(gt_leg_gj_points)
    
    add_section_and_points(curr_map, entry["temporal_control"]["android"], loc_df_label, "android", "orange", with_points)
    add_section_and_points(curr_map, entry["temporal_control"]["ios"], loc_df_label, "ios", "purple", with_points)
    
    curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
    folium.LayerControl().add_to(curr_map)
    return curr_map

In [ ]:
map_list = [display_gt_and_controls(e, "location_df") for e in deep_dive_dataset.values()]; len(map_list)

In [ ]:
evaluation_maps = bre.Figure()
for i, curr_map in enumerate(map_list):
    evaluation_maps.add_subplot(2, 2, i+1).add_child(curr_map)
evaluation_maps

Features of interest in the maps:
- `easiest`: android and ios match beautifully with ground truth
- `weird_android`: ios matches with ground truth, but android (orange) has inconsistencies near the start (triangle) and deviates a couple of other times in the middle
- `temporal_zigzags`: the ios data is much less accurate, with loops around San Bruno and Daly City and a mid-bay zigzag. However, it is much more complete, extending almost all the way to Berkeley. The android trajectory seems to be closer to the ground truth, but abruptly ends around Duboce/Mission and does not continue during this section
- `backtracking`: intentional backtracking at the foot of Coleman Ave

### Reference trajectory creation components

Given these errors in the control trajectories, we now consider multiple potential algorithms for converting them to reference trajectories. These algorithms represent a tradeoff between correctness and usability. For example, we could choose a really conservative algorithm which would be very likely to be close to ground truth, but only few runs might result in usable reference trajectories.

#### Normalization

Before we compare temporal trajectories, we need to normalize them. This is because the control data is sensed from two separate phones, and the timestamps are different. So we need to align the timestamp trajectories so that we can see how far apart they are. We use a simple normalization in which we linearly interpolate the lat/lng separately at a set of common timestamps. 

**Note**: we have to restrict the common timestamps to the range of input locations for each trajectory. This is because interpolation does not work well, and in cases where the trajectory is incomplete (`temporal_zigzags`) the generated location for the entire missing timestamp range is the final point. We ensure that the timestamps across phone still align by converting them to ints.

#### Distances and projections

In some of the more complex comparisons, we will distance comparisons between trajectories. We use two kinds of potential distances:
1. `projection`: which is the projection of one trajectory on the other
1. `distance`: which is the distance between the two trajectories at a particular point

The comparisons can be between the control trajectories, or between the control trajectories and ground truth

#### Filtering

After computing the distances, we can filter out obviously invalid points. Some validity checks are:

1. `control_trajectories`: points are valid if they are close to each other
1. `ground_truth_both`: points are valid if they are close to the ground truth in both trajectories
1. `ground_truth_either`: points are valid if they are close to the ground truth in either trajectory
1. `distance_along_route`: points are valid if they are further along the ground truth trajectory 

#### Merging

After we compare the trajectories and decide which points to retain, we will potentially have two fairly close-by points. We need to figure out how to combine them into one. Potential options are:
1. random (requires only control trajectories)
1. midpoint (requires only control trajectories)
1. closer to ground truth (requires control trajectories and ground truth)
1. closer to previous distance along route (requires control trajectories and ground truth)

### Candidate algorithms

We combine the components above into the following potential algorithms

#### Control trajectory-only comparisons

Note that in this case, because we interpolate for different ranges, we need to ensure that we are calculating the diff between the same points and not different points. See `train_bus_ebike_mtv_ucb/berkeley_to_mtv_SF_express_bus/ebike_bikeshare_urban_long_1` for an example. The section started at `2019-07-25T16:36:32.386274-07:00`, but on iOS, we did not get any points until 16:36:41, although we had android points right from 16:36:33. This means that we were matching up points at different timestamps, which caused the distance to be larger.

In [ ]:
pv_ucb.map()["ios"]["ucb-sdb-ios-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][2]["location_df"].query("ts > (1564097792 - 15) & ts < (1564097792 + 15)").fmt_time

In [ ]:
pv_ucb.map()["android"]["ucb-sdb-android-1"]["evaluation_ranges"][1]["evaluation_trip_ranges"][2]["location_df"].query("ts > (1564097792 - 15) & ts < (1564097792 + 15)").fmt_time

In [ ]:
def display_gt_and_reference(entry, loc_df_label, with_points=False):
    curr_map = folium.Map()
    # print("Using ground truth %s" % entry["ground_truth"]["leg"]["id"])
    gt_leg_gj = eisd.SpecDetails.get_geojson_for_leg(entry["ground_truth"]["leg"])
    gt_leg_gj_feature = folium.GeoJson(gt_leg_gj, name="ground_truth")
    curr_map.add_child(gt_leg_gj_feature)
    if with_points:
        gt_leg_gj_points = ezpv.get_point_markers(gt_leg_gj[2], name="ground_truth_points", color="green")
        curr_map.add_child(gt_leg_gj_points)
    
    sensed_location_df = entry[loc_df_label]
    # print("Adding section for %s with length %s" % (loc_df_label, len(sensed_location_df)))
    if len(sensed_location_df) > 0:
        sensed_section_gj = gj.Feature(geometry=gj.LineString(coordinates=list(zip(sensed_location_df.longitude, sensed_location_df.latitude))),
                                   properties={"style": {"color": "red"}, "ts": list(sensed_location_df.ts)})
        sensed_leg_gj_feature = folium.GeoJson(sensed_section_gj, name="reference_trajectory")
        curr_map.add_child(sensed_leg_gj_feature)
        if with_points:
            sensed_leg_gj_points = ezpv.get_point_markers(sensed_section_gj, name="reference_points", color="red", tz="America/Los_Angeles")
            curr_map.add_child(sensed_leg_gj_points)
    
    curr_map.fit_bounds(gt_leg_gj_feature.get_bounds())
    folium.LayerControl().add_to(curr_map)
    return curr_map

In [ ]:
R = 40075017 # circumference of the earth in meters

def ref_ct_general(e, b_merge_fn, dist_threshold, tz="UTC"):
    new_location_df_a = emr.get_int_aligned_trajectory(e["temporal_control"]["android"]["location_df"], tz)
    new_location_df_i = emr.get_int_aligned_trajectory(e["temporal_control"]["ios"]["location_df"], tz)
    merged_df = pd.merge(new_location_df_a, new_location_df_i, on="ts",
        how="inner", suffixes=("_a", "_i")).sort_values(by="ts", axis="index")
    merged_df["t_distance"] = gpd.GeoSeries(merged_df.geometry_a).distance(gpd.GeoSeries(merged_df.geometry_i)) * (R/360)
    filtered_merged_df = merged_df.query("t_distance < @dist_threshold")
    print("After filtering, retained %d of %d (%s)" % 
          (len(filtered_merged_df), max(len(new_location_df_a), len(new_location_df_i)),
            (len(filtered_merged_df)/max(len(new_location_df_a), len(new_location_df_i)))))
    merge_fn = functools.partial(emr.collapse_inner_join, b_merge_fn=b_merge_fn)
    initial_reference_gpdf = gpd.GeoDataFrame(list(filtered_merged_df.apply(merge_fn, axis=1)))
    print(initial_reference_gpdf.columns)
    if len(initial_reference_gpdf.columns) > 1:
        initial_reference_gpdf["fmt_time"] = initial_reference_gpdf.ts.apply(lambda ts: arrow.get(ts).to(tz))
        assert len(initial_reference_gpdf[initial_reference_gpdf.latitude.isnull()]) == 0, "Found %d null entries out of %d total" % (len(initial_reference_gpdf.latitude.isnull()), len(initial_reference_gpdf))
        return initial_reference_gpdf
    else:
        return gpd.GeoDataFrame()

##### Effect of threshold

For the "good" trajectories (`easiest` and `backtracking`), the threshold has essentially no effect. The trajectories are so good that the tolerance is less than 10 meters and we retain the entire trajectory. For the other two, even if the coverage is good, the quality of the final trajectory is not that great. For example, consider the offset along Hawthorne Ave in the `weird_android` case and the deviations from the ground truth along BART in the `temporal_zigzags` case.

And for the `temporal_zigzags`, at small thresholds, almost nothing matches.

Also, in the case where the data was missing from one of the trajectories (i.e. `temporal_zigzags` on `android`), our reference trajectory also ends, leading to poor coverage compared to the alternatives.

In [ ]:
for e in deep_dive_dataset.values():
    e["ct_midpoint_loc_df_10"] = ref_ct_general(e, emr.b_merge_midpoint,
                                               10, tz="America/Los_angeles")
    e["ct_midpoint_loc_df_25"] = ref_ct_general(e, emr.b_merge_midpoint,
                                               25, tz="America/Los_angeles")
    e["ct_midpoint_loc_df_50"] = ref_ct_general(e, emr.b_merge_midpoint,
                                               50, tz="America/Los_angeles")
    e["ct_midpoint_loc_df_100"] = ref_ct_general(e, emr.b_merge_midpoint,
                                               100, tz="America/Los_angeles")

In [ ]:
evaluation_maps = bre.Figure()
test_candidates = ["ct_midpoint_loc_df_10", "ct_midpoint_loc_df_25", "ct_midpoint_loc_df_50", "ct_midpoint_loc_df_100"]
nRows = len(test_candidates)
for i, c in enumerate(test_candidates):
    evaluation_maps.add_child(bre.Html(c))
    map_list = [display_gt_and_reference(e, c) for e in deep_dive_dataset.values()]    
    for j, curr_map in enumerate(map_list):
        evaluation_maps.add_subplot(nRows, 4, 4*i+(j+1)).add_child(curr_map)
evaluation_maps

##### Effect of merge function

Again, for the "good" trajectories (`easiest` and `backtracking`), the merge function has essentially no effect. The trajectories are so good that the tolerance is less than 10 meters and using the midpoint versus random versus closest makes no difference.

For the other two, choosing the closest point allows us to handle the case where the errors are related primarily to one phone (e.g. in the `weird_android` case). So if the iOS trajectory is good (as determined by a comparison with spatial ground truth), we can use it directly and not have it be polluted by the bad android trajectory.

So effectively, with this approach, we are choosing the good points from each of the trajectories and stitching them together to create an overall trajectory. This implies that we cannot use an inner join for the merge, since then we would only retain points that were good in **both** trajectories. We use an outer join instead, and if there are points that are good only in one trajectory, we just select them.

Note that this allows us to use much smaller thresholds and still get coverages similar to the higher thresholds while using midpoints

However, stitching together points from multiple trajectories is challenging if there are time sychronization issues. While the timestamps for a single phone are guaranteed to be consistent, timestamps across phones are not guaranteed to be consistent, and flip flopping between phones can lead to zigzags in the final trajectories. For example, in the `weird_android` case, see:
- the section of Springer between Todd and El Monte,
- the intersection of El Camino and Shoreline,
- El Monte near Mayer Court

In [ ]:
def ref_gt_general(e, b_merge_fn, dist_threshold,
        tz="UTC"):
    new_location_df_a = emr.get_int_aligned_trajectory(e["temporal_control"]["android"]["location_df"], tz)
    new_location_df_i = emr.get_int_aligned_trajectory(e["temporal_control"]["ios"]["location_df"], tz)
    emr.fill_gt_linestring(e)
    gt_linestring = e["ground_truth"]["linestring"]
    emr.add_gt_error_projection(new_location_df_a, gt_linestring)
    emr.add_gt_error_projection(new_location_df_i, gt_linestring)
    filtered_location_df_a = new_location_df_a.query("gt_distance < @dist_threshold")
    filtered_location_df_i = new_location_df_i.query("gt_distance < @dist_threshold")
    print("After filtering, %d of %d (%s) for android and %d of %d (%s) for ios" % 
          (len(filtered_location_df_a), len(new_location_df_a), (len(filtered_location_df_a)/len(new_location_df_a)),
           len(filtered_location_df_i), len(new_location_df_i), (len(filtered_location_df_i)/len(new_location_df_i))))
    merged_df = pd.merge(filtered_location_df_a, filtered_location_df_i, on="ts",
        how="outer", suffixes=("_a", "_i")).sort_values(by="ts", axis="index")
    merge_fn = functools.partial(emr.collapse_outer_join_stateless, b_merge_fn=b_merge_fn)
    initial_reference_gpdf = gpd.GeoDataFrame(list(merged_df.apply(merge_fn, axis=1)))
    if len(initial_reference_gpdf.columns) > 1:
        initial_reference_gpdf["fmt_time"] = initial_reference_gpdf.ts.apply(lambda ts: arrow.get(ts).to(tz))
        print("After merging, found %d of android %d (%s), ios %d (%s)" % 
              (len(initial_reference_gpdf), len(new_location_df_a), (len(initial_reference_gpdf)/len(new_location_df_a)),
               len(new_location_df_i), (len(initial_reference_gpdf)/len(new_location_df_i))))
        assert len(initial_reference_gpdf[initial_reference_gpdf.latitude.isnull()]) == 0, "Found %d null entries out of %d total" % (len(initial_reference_gpdf.latitude.isnull()), len(initial_reference_gpdf))
        return initial_reference_gpdf
    else:
        return gpd.GeoDataFrame()

In [ ]:
for e in deep_dive_dataset.values():
    e["ct_closer_gt_dist_10"] = ref_gt_general(e, emr.b_merge_closer_gt_dist,
                                               10, tz="America/Los_angeles")
    e["ct_closer_gt_dist_25"] = ref_gt_general(e, emr.b_merge_closer_gt_dist,
                                               25, tz="America/Los_angeles")

In [ ]:
evaluation_maps = bre.Figure()
test_candidates = ["ct_closer_gt_dist_10", "ct_closer_gt_dist_25", "ct_midpoint_loc_df_50"]
nRows = len(test_candidates)
for i, c in enumerate(test_candidates):
    evaluation_maps.add_child(bre.Html(c))
    map_list = [display_gt_and_reference(e, c) for e in deep_dive_dataset.values()]    
    for j, curr_map in enumerate(map_list):
        evaluation_maps.add_subplot(nRows, 4, 4*i+(j+1)).add_child(curr_map)
evaluation_maps

##### Incorporating _forward motion_

We can handle the time synchronization issues by ensuring that travel only occurs "forward" along the ground truth trajectory. We can determine the distance travelled so far as a projection of the control trajectory along the ground truth and only incorporate points that make forward progress. Note that this requires the use of a global variable to track the distance so far, and the merging function is no longer stateless.

And, as feared, in the place where there is a genuine backtrack (`backtracking` use case at Market and Devine), we trim some of the backtrack.

In [ ]:
importlib.reload(emr)

In [ ]:
def ref_travel_forward(e, dist_threshold, tz="UTC"):
    # This function needs a global variable
    emr.distance_so_far = 0
    new_location_df_a = emr.get_int_aligned_trajectory(e["temporal_control"]["android"]["location_df"], tz)
    new_location_df_i = emr.get_int_aligned_trajectory(e["temporal_control"]["ios"]["location_df"], tz)
    emr.fill_gt_linestring(e)
    gt_linestring = e["ground_truth"]["linestring"]
    emr.add_gt_error_projection(new_location_df_a, gt_linestring)
    emr.add_gt_error_projection(new_location_df_i, gt_linestring)
    new_location_df_a["gt_cum_proj"] = new_location_df_a.gt_projection.cumsum()
    new_location_df_i["gt_cum_proj"] = new_location_df_i.gt_projection.cumsum()
    filtered_location_df_a = new_location_df_a.query("gt_distance < @dist_threshold")
    filtered_location_df_i = new_location_df_i.query("gt_distance < @dist_threshold")
    print("After filtering, %d of %d (%s) for android and %d of %d (%s) for ios" % 
          (len(filtered_location_df_a), len(new_location_df_a), (len(filtered_location_df_a)/len(new_location_df_a)),
           len(filtered_location_df_i), len(new_location_df_i), (len(filtered_location_df_i)/len(new_location_df_i))))
    merged_df = pd.merge(filtered_location_df_a, filtered_location_df_i, on="ts",
        how="outer", suffixes=("_a", "_i")).sort_values(by="ts", axis="index")
    merge_fn = functools.partial(emr.collapse_outer_join_dist_so_far, more_details_fn = None)
    initial_reference_gpdf = gpd.GeoDataFrame(list(merged_df.apply(merge_fn, axis=1)))
    if len(initial_reference_gpdf.columns) > 1:
        initial_reference_gpdf["fmt_time"] = initial_reference_gpdf.ts.apply(lambda ts: arrow.get(ts).to(tz))
        reference_gpdf = initial_reference_gpdf[initial_reference_gpdf.latitude.notnull()]
        print("After merging, found %d / %d of android %d (%s), ios %d (%s)" % 
              (len(reference_gpdf), len(initial_reference_gpdf), len(new_location_df_a), (len(reference_gpdf)/len(new_location_df_a)),
               len(new_location_df_i), (len(reference_gpdf)/len(new_location_df_i))))
        assert len(reference_gpdf[reference_gpdf.latitude.isnull()]) == 0, "Found %d null entries out of %d total" % (len(reference_gpdf[reference_gpdf.latitude.isnull()]), len(initial_reference_gpdf))
        return reference_gpdf
    else:
        return gpd.GeoDataFrame()

In [ ]:
for e in deep_dive_dataset.values():
    e["travel_forward_10"] = ref_travel_forward(e, 10, tz="America/Los_angeles")
    e["travel_forward_25"] = ref_travel_forward(e, 25, tz="America/Los_angeles")
    e["travel_forward_50"] = ref_travel_forward(e, 50, tz="America/Los_angeles")

In [ ]:
evaluation_maps = bre.Figure()
test_candidates = ["travel_forward_10", "travel_forward_25", "travel_forward_50"]
nRows = len(test_candidates)
for i, c in enumerate(test_candidates):
    evaluation_maps.add_child(bre.Html(c))
    map_list = [display_gt_and_reference(e, c) for e in deep_dive_dataset.values()]    
    for j, curr_map in enumerate(map_list):
        evaluation_maps.add_subplot(nRows, 4, 4*i+(j+1)).add_child(curr_map)
evaluation_maps

### Estimating the coverage of reference trajectories

- The `ct_midpoint` methods do not require ground truth, do not generate zigzags and work well with backtracking. Their main drawback is that they work well only for very high quality trajectories.
- The `forward_motion` method works well even with poor quality trajectories and gives great coverage but does not handle backtracking

An ensemble of the two is likely to be a successful method overall. Let's estimate how this will work statistically for the trajectories from our dataset

In [ ]:
def get_gt_linestring(gt_leg):
    """
    Get lat-long corrdinates in ground truth
    """
    if 'route_coords' in gt_leg:
        coords = gt_leg['route_coords']['geometry']['coordinates']
    else:
        coords = []
    return shp.geometry.LineString(coords)

In [ ]:
def get_reference_trajectory_tree(pv):
    ref_tree = {}
    # This is a GeoSeries
    # polygons = get_polygons(pv)
    
    for phone_os, phone_map in pv.map().items():
        for phone_label, phone_detail_map in phone_map.items():
            for (r_idx, r) in enumerate(phone_detail_map["evaluation_ranges"]):
                if r["eval_role_base"] != "accuracy_control":
                    continue
                for (tr_idx, tr) in enumerate(r["evaluation_trip_ranges"]):
                    ref_travel_forward
                    for (sr_idx, sr) in enumerate(tr["evaluation_section_ranges"]):
                        # This is a Shapely LineString
                        section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"])
                        section_gt_points = get_gt_linestring(section_gt_leg)
                        if section_gt_points.is_empty:
                            print("No ground truth route for %s %s, must be polygon, skipping..." % (tr["trip_id_base"], sr["trip_id_base"]))
                            assert section_gt_leg["type"] != "TRAVEL", "For %s, %s, %s, %s, %s found type %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx, section_gt_leg["type"])
                            continue
                        if len(sr['location_df']) == 0:
                            print("No sensed locations found, role = %s skipping..." % (r["eval_role_base"]))
                            # assert r["eval_role_base"] == "power_control", "Found no locations for %s, %s, %s, %s, %s" % (phone_os, phone_label, r_idx, tr_idx, sr_idx)
                            continue
                        
                        sec_name = tr["trip_id_base"] + "/" + sr["trip_id_base"] + "_" + str(r_idx)
                        if sec_name not in ref_tree:
                            ref_tree[sec_name] = {
                                "trip_id": tr["trip_id_base"],
                                "section_id": sr["trip_id_base"],
                                "run": r_idx,
                                "ground_truth": {
                                    "leg": section_gt_leg,
                                    "linestring": section_gt_points
                                }
                            }
                        
                        assert sec_name in ref_tree
                        e = ref_tree[sec_name]
                        # This is a GeoDataFrame
                        # section_measured_points = get_travel_trajectory(sr['location_df'], polygons)
                        section_measured_points = sr["location_df"]
                        if "temporal_control" not in e:
                            e["temporal_control"] = {}
                            e["start_ts"] = sr["start_ts"]
                            e["end_ts"] = sr["end_ts"]
                        e["temporal_control"][phone_os] = sr
    return ref_tree
                        
def fill_ref_tree_entry(e):
    print("Considering entry %s %s %s" % (e["trip_id"], e["section_id"], e["run"]))
    curr_tz = "America/Los_angeles"
    assert "android" in e["temporal_control"] and "ios" in e["temporal_control"]
    e["travel_forward_25"] = ref_travel_forward(e, 25, tz=curr_tz)
    e["travel_forward_50"] = ref_travel_forward(e, 50, tz=curr_tz)
    e["ct_midpoint_25"] = ref_ct_general(e, emr.b_merge_midpoint, 25, tz=curr_tz)
    e["ct_midpoint_50"] = ref_ct_general(e, emr.b_merge_midpoint, 50, tz=curr_tz)

def get_coverage(pv, ref_tree):
    spatial_error_list = []
    for e in ref_tree.values():
        # we resample every second, so full coverage would involve 1 entry for each second in the section
        # If we have less than that, the length of the dataframe will be less
        coverage = lambda df, sr: len(df)/(sr["end_ts"] - sr["start_ts"])
        spatial_error_base = {"timeline": pv.spec_details.curr_spec["id"],
                         "trip_id": e["trip_id"], "section_id": e["section_id"], "run": e["run"]}
        
        for result in ["travel_forward_25", "travel_forward_50", "ct_midpoint_25", "ct_midpoint_50"]:
            spatial_error = copy.copy(spatial_error_base)
            spatial_error["algo"] = result
            spatial_error["coverage"] = coverage(e[result], e)
            spatial_error_list.append(spatial_error)
    return spatial_error_list

def get_reference_coverage(pv, ref_tree_root):
    ref_tree = get_reference_trajectory_tree(pv)
    ref_tree_root[pv.spec_details.curr_spec["id"]] = ref_tree
    [fill_ref_tree_entry(e) for e in ref_tree.values()]
    return get_coverage(pv, ref_tree)

In [ ]:
ref_tree = {}
coverage_list = []
coverage_list.extend(get_reference_coverage(pv_la, ref_tree))
coverage_list.extend(get_reference_coverage(pv_sj, ref_tree))
coverage_list.extend(get_reference_coverage(pv_ucb, ref_tree))

coverage_df = pd.DataFrame(coverage_list)

In [ ]:
coverage_df.head()

In [ ]:
timeline_list = ["car_scooter_brex_san_jose", "unimodal_trip_car_bike_mtv_la"]
test_candidates = ["ct_midpoint_25", "ct_midpoint_50", "travel_forward_25", "travel_forward_50"]
for i, tl in enumerate(timeline_list):
    unique_sections = pd.Series(coverage_df.query("timeline == @tl").section_id.unique())
    ifig, ax_array = plt.subplots(nrows=1,ncols=len(unique_sections),figsize=(12,3), sharex=False, sharey=True)
    for j, sid in enumerate(unique_sections):
        coverage_df.query("timeline == @tl & section_id == @sid").boxplot(ax = ax_array[j], column=["coverage"], by=["algo"], showbox=False, whis="range")
        ax_array[j].set_title(sid)
        ax_array[j].set_xticklabels(["ctm_25", "ctm_50", "tf_25", "tf_50"])
        ax_array[j].set_xlabel("")
    ifig.suptitle(tl, y=1.05)

In [ ]:
long_timeline = "train_bus_ebike_mtv_ucb"
test_candidates = ["ct_midpoint_25", "ct_midpoint_50", "travel_forward_25", "travel_forward_50"]

unique_sections = pd.Series(coverage_df.query("timeline == @long_timeline").section_id.unique())
nRows = len(unique_sections)/3 + 1
ifig = plt.Figure(figsize=(15,15))
ax_list = []
first_ax = None
for j, sid in enumerate(unique_sections):
    if first_ax is None:
        curr_ax = ifig.add_subplot(nRows, 3, j+1)
        first_ax = curr_ax        
    else:
        curr_ax = ifig.add_subplot(nRows, 3, j+1, sharex=first_ax, sharey=first_ax)
    ax_list.append(curr_ax)
    coverage_df.query("timeline == @long_timeline & section_id == @sid").boxplot(ax = curr_ax, column=["coverage"], by=["algo"], showbox=False, whis="range")
    curr_ax.set_title(sid)
    curr_ax.set_xlabel("")
    curr_ax.tick_params(labelbottom=False)
    curr_ax.tick_params(labelleft=False)
    
for i in range(1,4):
    ax_list[-i].tick_params(labelbottom=True)
    ax_list[-i].set_xticklabels(["ctm_25", "ctm_50", "tf_25", "tf_50"])
    
for i in range(0,len(ax_list),3):
    print("Displaying y tick labels for %s" % i)
    ax_list[i].tick_params(labelleft=True)

ifig.suptitle(long_timeline)
ifig

In [ ]:
outliers = coverage_df.query("section_id == 'freeway_driving_weekday' & (algo == 'travel_forward_25' | algo == 'ct_midpoint_25')"); outliers

### Outlier verification


#### travel_forward_25 	0.565114 	1 	commuter_rail_with_tunnels 	train_bus_ebike_mtv_ucb 	berkeley_to_mtv_SF_express_bus

In this case, the android trajectory is correct, but the iOS trajectory is all over the place, with lots of zigzags, gaps and other wierdnesses.

In [ ]:
sel_name = "berkeley_to_mtv_SF_express_bus/commuter_rail_with_tunnels_1"
e = ref_tree["train_bus_ebike_mtv_ucb"][sel_name]
fig = bre.Figure()
fig.add_subplot(1,3,1).add_child(display_gt_and_reference(e, "travel_forward_25", with_points=True))
fig.add_subplot(1,3,2).add_child(display_gt_and_reference(e, "ct_midpoint_25", with_points=True))
fig.add_subplot(1,3,3).add_child(display_gt_and_controls(e, "location_df", with_points=True))

#### travel_forward_25 	0.166812 	2 	walk to the bikeshare location 	train_bus_ebike_mtv_ucb 	berkeley_to_mtv_SF_express_bus

This is because of walking around to find the bicycle, and the fact that we don't strip out the points within the polygons any more

In [ ]:
sel_name = "berkeley_to_mtv_SF_express_bus/walk to the bikeshare location_2"
e = ref_tree["train_bus_ebike_mtv_ucb"][sel_name]
fig = bre.Figure()
fig.add_subplot(1,2,1).add_child(display_gt_and_reference(e, "travel_forward_25", with_points=True))
fig.add_subplot(1,2,2).add_child(display_gt_and_reference(e, "ct_midpoint_25", with_points=True))

#### travel_forward_25 	0.836886 	4 	walk_back_from_bus 	car_scooter_brex_san_jose 	bus trip with e-scooter access

Looks fine; unsure why this is so low

In [ ]:
sel_name = "bus trip with e-scooter access/walk_back_from_bus_4"
e = ref_tree["car_scooter_brex_san_jose"][sel_name]
fig = bre.Figure()
fig.add_subplot(1,2,1).add_child(display_gt_and_reference(e, "travel_forward_25", with_points=True))
fig.add_subplot(1,2,2).add_child(display_gt_and_reference(e, "ct_midpoint_25", with_points=True))

#### travel_forward_25 	0.276315 	2 	light_rail_below_above_ground 	train_bus_ebike_mtv_ucb 	berkeley_to_mtv_SF_express_bus

In this case, there is a time mismatch, so adjacent points are separated by ~ 6 seconds.
e.g. at around Embarcadero and Bryant, the android point is `2019-07-26 18:15:18` while the iOS point is `2019-07-26 18:15:24`. The android point that matches `2019-07-26 18:15:24` is halfway down the block.

In [ ]:
sel_name = "berkeley_to_mtv_SF_express_bus/light_rail_below_above_ground_2"
e = ref_tree["train_bus_ebike_mtv_ucb"][sel_name]
fig = bre.Figure()
fig.add_subplot(1,3,1).add_child(display_gt_and_reference(e, "travel_forward_25", with_points=True))
fig.add_subplot(1,3,2).add_child(display_gt_and_reference(e, "ct_midpoint_25", with_points=True))
fig.add_subplot(1,3,3).add_child(display_gt_and_controls(e, "location_df", with_points=True))

In [ ]:
# travel_forward_25 	0.350489 	1 	walk_to_bus 	train_bus_ebike_mtv_ucb 	mtv_to_berkeley_sf_bart
sel_name = "mtv_to_berkeley_sf_bart/walk_to_bus_1"
e = ref_tree["train_bus_ebike_mtv_ucb"][sel_name]
fig = bre.Figure()
fig.add_subplot(1,3,1).add_child(display_gt_and_reference(e, "travel_forward_25", with_points=True))
fig.add_subplot(1,3,2).add_child(display_gt_and_reference(e, "ct_midpoint_25", with_points=True))
fig.add_subplot(1,3,3).add_child(display_gt_and_controls(e, "location_df", with_points=True))

In [ ]:
# travel_forward_25 	0.845444 	0 	freeway_driving_weekday 	car_scooter_brex_san_jose 	freeway_driving_weekday
sel_name = "freeway_driving_weekday/freeway_driving_weekday_0"
e = ref_tree["car_scooter_brex_san_jose"][sel_name]
fig = bre.Figure()
fig.add_subplot(1,3,1).add_child(display_gt_and_reference(e, "travel_forward_25", with_points=True))
fig.add_subplot(1,3,2).add_child(display_gt_and_reference(e, "ct_midpoint_25", with_points=True))
fig.add_subplot(1,3,3).add_child(display_gt_and_controls(e, "location_df", with_points=True))

In [ ]:
def get_centroid(curr_map):
    bbox = []
    [bbox.extend(reversed(p)) for p in curr_map.get_bounds()]
    return shp.geometry.box(*bbox).centroid